In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def gauss_newton(y, H):
    x = np.linalg.solve(H.T@H,H.T@y)
    return x

In [ ]:
def huber(x,l):
    if np.abs(x) <= l:
        return 0.5*x**2
    else:
        return l*(np.abs(x)-0.5*l)

In [ ]:
def lev_marq(H,y):
    trust = 1e-3
    hessian = H.T@H + trust*np.diag(H.T@H)
    x = np.linalg.solve()

In [ ]:
def robust_gauss_newton(y,H):
    x = np.linalg.solve(H.T@H,H.T@y)
    return x

In [ ]:
m = 10
n = 2
X = np.array([[1.],[1.]])
R = 1e-3*np.eye(m)
xi = np.reshape(np.arange(0,1,1/m,dtype=float),(m,1))
H = np.concatenate((xi,np.ones((m,1))),axis=1)
y = H@X + np.linalg.cholesky(R)@np.random.randn(m,1)
y[2,0] += 1
x_hat = gauss_newton(y,H)

In [ ]:
x_plot = np.arange(0, 1, 0.01)
y_plot = x_hat[0,0]*x_plot+x_hat[1,0]
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot,'r')
plt.scatter(xi,y)
print(x_hat)